In [8]:
import numpy as np
import os
import pandas as pd

In [24]:
import requests
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime
import os
from bs4 import BeautifulSoup
import re

# Base URL for the WSA-Enlil data
base_url = "https://data.ngdc.noaa.gov/earth-science-services/models/space-weather/wsa-enlil/"

# Function to get available data directories
def get_available_directories(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find directories (typically they end with '/')
        directories = [link.get('href') for link in soup.find_all('a') 
              if link.get('href').endswith('/')
              and not link.get('href').startswith('..') 
              and not link.get('href').startswith('http')
              and not link.get('href').startswith('/')]
        return directories
    else:
        print(f"Failed to access {url}, status code: {response.status_code}")
        return []

# Function to download a specific file
def download_file(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded: {save_path}")
        return True
    else:
        print(f"Failed to download {url}, status code: {response.status_code}")
        return False

# Get available model runs (usually organized by date)
yr_directories = get_available_directories(base_url)
print(f"Available directories: {yr_directories}")

# Example: Select the most recent directory and explore its contents
if yr_directories:
    # Sort directories to get the most recent one (assuming date-based naming)
    # latest_dir = sorted(directories)[-1]
    # latest_url = base_url + latest_dir

    # Loop over all years.
    years = [yr_directories[-1]]

    for yr in years:
        mo_directories = get_available_directories(base_url+yr)
        months = [mo_directories[-1]]

        for mo in months:
            mo_url = base_url+yr+mo
            response = requests.get(mo_url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                # Find netCDF files (common format for WSA-Enlil data)
                nc_files = [link.get('href') for link in soup.find_all('a') 
                           if link.get('href').endswith('.nc')]
                
                if nc_files:
                    # Download and load the first netCDF file as an example
                    example_file = nc_files[0]
                    file_url = latest_url + example_file
                    local_path = example_file
                    
                    if download_file(file_url, local_path):
                        # Load the netCDF file using xarray
                        ds = xr.open_dataset(local_path)
                        
                        # Display basic information about the dataset
                        print("\nDataset Information:")
                        print(ds.info())
                        
                        # Example: Plot a variable if available
                        if 'bz' in ds.variables:
                            plt.figure(figsize=(10, 6))
                            ds['bz'].plot()
                            plt.title('Bz Component of Magnetic Field')
                            plt.xlabel('Time')
                            plt.ylabel('Bz (nT)')
                            plt.show()
                        
                        # Clean up - remove downloaded file
                        os.remove(local_path)
                else:
                    print(f"No netCDF files found in the latest directory: {mo_url}")
            else:
                print(f"Failed to access month directory: {mo_url}")
    # else:
    #     print(f"Failed to access year directory: {latest_url}")
else:
    print("No directories found.")

Available directories: ['2011/', '2012/', '2013/', '2014/', '2015/', '2016/', '2017/', '2018/', '2019/', '2020/', '2021/', '2022/', '2023/', '2024/', '2025/']
No netCDF files found in the latest directory: https://data.ngdc.noaa.gov/earth-science-services/models/space-weather/wsa-enlil/2025/11/


In [41]:
def read_nc(file):
    # Load the netCDF file using xarray
    ds = xr.open_dataset(file)
    
    # Display basic information about the dataset
    print("\nDataset Information:")
    print(ds.info())
    
    # Example: Plot a variable if available
    if 'bz' in ds.variables:
        plt.figure(figsize=(10, 6))
        ds['bz'].plot()
        plt.title('Bz Component of Magnetic Field')
        plt.xlabel('Time')
        plt.ylabel('Bz (nT)')
        plt.show()
    return ds


# swpc_wsaenlil_bkg_20251004_0000
yr = 2025
mo = 10
da = 4

data_path = os.path.join("Data", f"swpc_wsaenlil_bkg_{yr:02d}{mo:02d}{da:02d}_0000")
if os.path.exists(data_path):
    data_file = os.path.join(data_path, "wsa_enlil.mrid00000000.suball.nc")
    if os.path.exists(data_file):
        # Open .nc data file.
        ds = read_nc(data_file)
        
    else:
        print("Data file does not exist:", data_file)
else:
    print("Data path does not exist:", data_path)


Dataset Information:
xarray.Dataset {
dimensions:
	x = 512 ;
	y = 60 ;
	z = 180 ;
	t = 169 ;
	earth_t = 13166 ;

variables:
	float32 x_coord(x) ;
		x_coord:long_name = radial cell positions ;
		x_coord:units = m ;
	float32 y_coord(y) ;
		y_coord:long_name = co-latitude cell positions ;
		y_coord:units = radians ;
	float32 z_coord(z) ;
		z_coord:long_name = longitude cell positions ;
		z_coord:units = radians ;
	timedelta64[ns] time(t) ;
		time:long_name = time relative to REFDATE ;
	int16 dd12_3d(t, y, x) ;
		dd12_3d:dd12_max = 1.9698555621544462e-18 ;
		dd12_3d:dd12_min = 1.0295863164226942e-21 ;
		dd12_3d:long_name = uncalibrated plasma density in rad-colat-time, longitude zero ;
		dd12_3d:units = kg/m3 - after calibration ;
	int16 vv12_3d(t, y, x) ;
		vv12_3d:vv12_max = 728108.6875 ;
		vv12_3d:vv12_min = 240771.671875 ;
		vv12_3d:long_name = uncalibrated velocity in rad-colat-time, longitude zero ;
		vv12_3d:units = m/s - after calibration ;
	int16 pp12_3d(t, y, x) ;
		pp12_3d:pp12

C:\Users\haley\AppData\Local\Temp\ipykernel_53396\4217810667.py:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset(file)


In [50]:
for var in list(ds):
    print(var, "\t", ds[var].attrs['long_name'])

x_coord 	 radial cell positions
y_coord 	 co-latitude cell positions
z_coord 	 longitude cell positions
time 	 time relative to REFDATE
dd12_3d 	 uncalibrated plasma density in rad-colat-time, longitude zero
vv12_3d 	 uncalibrated velocity in rad-colat-time, longitude zero
pp12_3d 	 uncalibrated magnetic polarity in rad-colat-time, longitude zero
dd13_3d 	 uncalibrated plasma density in rad-long-time, Earth latitude
vv13_3d 	 uncalibrated velocity in rad-long-time, Earth latitude
pp13_3d 	 uncalibrated magnetic polarity in rad-long-time, Earth latitude
dd23_3d 	 uncalibrated plasma density in colat-long-time, 1AU surface
vv23_3d 	 uncalibrated velocity in colat-long-time, 1AU surface
pp23_3d 	 uncalibrated magnetic polarity in colat-long-time, 1AU surface
Earth_TIME 	 relative time of Earth and STEREO time series data
Earth_X1 	 Earth radial position
Earth_X2 	 Earth co-latitude position
Earth_X3 	 Earth longitude position
Earth_Density 	 plasma density at the position of Earth
Earth_T

In [45]:
len(ds['x_coord']) # alt

512

In [46]:
len(ds['y_coord']) # colat

60

In [47]:
len(ds['z_coord']) # lon

180

In [50]:
len(ds['time'])

169

In [49]:
len(ds['vv23_3d'])

169

In [51]:
ds['vv23_3d']

<xarray.DataArray 'vv23_3d' (t: 169, z: 180, y: 60)> Size: 4MB
[1825200 values with dtype=int16]
Dimensions without coordinates: t, z, y
Attributes:
    vv23_max:   731353.5
    vv23_min:   274298.97
    long_name:  uncalibrated velocity in colat-long-time, 1AU surface
    units:      m/s - after calibration

In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import folium
from folium.plugins import HeatMap

In [22]:
# # Method 1: Interactive Folium Map
# def create_folium_heatmap(data):
#     # Create a map centered at the mean of the data points
#     map_center = [data['Lat'].mean(), data['Lon'].mean()]
#     mymap = folium.Map(location=map_center, zoom_start=2, tiles='CartoDB positron')
    
#     # Prepare the data for the heatmap
#     heat_data = [[row['Lat'], row['Lon'], row['Plasma Density']] for _, row in data.iterrows()]
    
#     # Add the heatmap to the map
#     HeatMap(heat_data, radius=15, max_zoom=13).add_to(mymap)
    
#     # Save the map
#     mymap.save(os.path.join('Images', 'world_heatmap0.html'))
#     return mymap

# # Method 2: Static Matplotlib Map
# def create_matplotlib_heatmap(data):
#     plt.figure(figsize=(12, 8))
    
#     # Create a world map background
#     plt.scatter(
#         data['Lon'], 
#         data['Lat'],
#         c=data['Plasma Density'], 
#         cmap='hot_r',
#         alpha=0.7,
#         s=data['Plasma Density']*2,  # Size based on values
#         edgecolors='none'
#     )
    
#     # Add colorbar
#     cbar = plt.colorbar()
#     cbar.set_label('Plasma Density')
    
#     # Set plot limits and labels
#     plt.xlim(-180, 180)
#     plt.ylim(-90, 90)
#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.title('World Heatmap of Plasma Density')
    
#     # Add grid lines for reference
#     plt.grid(alpha=0.3)
    
#     # Save the figure
#     plt.savefig(os.path.join('Images', 'world_heatmap0.png'), dpi=300, bbox_inches='tight')
#     plt.show()


# altitudes_m = np.array(ds['Earth_X1']) # m
# colatitudes_rad = np.array(ds['Earth_X2']) # radians
# longitudes_rad = np.array(ds['Earth_X3']) # radians 

# # Convert to desired units.
# altitudes_km = altitudes_m / 1000
# colatitudes_deg = colatitudes_rad * 180/np.pi
# latitudes_deg = 90 - colatitudes_deg
# longitudes_deg = longitudes_rad * 180/np.pi

# grid_density = pd.DataFrame()
# grid_density['Lat'] = latitudes_deg
# grid_density['Lon'] = longitudes_deg
# grid_density['Alt'] = altitudes_km
# grid_density['Plasma Density'] = np.array(ds['Earth_Density'])
# grid_density.head()


# folium_map = create_folium_heatmap(grid_density)
# #create_matplotlib_heatmap(grid_density)

In [21]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib import cm
# import folium
# from folium.plugins import HeatMap

# # Sample data - replace this with your actual DataFrame
# # Assuming you have a DataFrame with columns: 'latitude', 'longitude', 'Values'
# # Creating sample data for demonstration
# np.random.seed(42)
# n_points = 500
# df = pd.DataFrame({
#     'latitude': np.random.uniform(-80, 80, n_points),
#     'longitude': np.random.uniform(-180, 180, n_points),
#     'Values': np.random.exponential(5, n_points)
# })

# # Method 1: Interactive Folium Map
# def create_folium_heatmap(data):
#     # Create a map centered at the mean of the data points
#     map_center = [data['latitude'].mean(), data['longitude'].mean()]
#     mymap = folium.Map(location=map_center, zoom_start=2, tiles='CartoDB positron')
    
#     # Prepare the data for the heatmap
#     heat_data = [[row['latitude'], row['longitude'], row['Values']] for _, row in data.iterrows()]
    
#     # Add the heatmap to the map
#     HeatMap(heat_data, radius=15, max_zoom=13).add_to(mymap)
    
#     # Save the map
#     mymap.save('world_heatmap.html')
#     return mymap

# # Method 2: Static Matplotlib Map
# def create_matplotlib_heatmap(data):
#     plt.figure(figsize=(12, 8))
    
#     # Create a world map background
#     plt.scatter(
#         data['longitude'], 
#         data['latitude'],
#         c=data['Values'], 
#         cmap='hot_r',
#         alpha=0.7,
#         s=data['Values']*2,  # Size based on values
#         edgecolors='none'
#     )
    
#     # Add colorbar
#     cbar = plt.colorbar()
#     cbar.set_label('Values')
    
#     # Set plot limits and labels
#     plt.xlim(-180, 180)
#     plt.ylim(-90, 90)
#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.title('World Heatmap of Values')
    
#     # Add grid lines for reference
#     plt.grid(alpha=0.3)
    
#     # Save the figure
#     plt.savefig('world_heatmap.png', dpi=300, bbox_inches='tight')
#     plt.show()

# # # Create both visualizations
# # folium_map = create_folium_heatmap(df)
# # create_matplotlib_heatmap(df)

# print("Heatmaps created! Check 'world_heatmap.html' for the interactive version.")

Heatmaps created! Check 'world_heatmap.html' for the interactive version.


In [102]:
# pip install tkinterweb

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 33.7 MB/s eta 0:00:00

   ---------------------------------------- 2/2 [tkinterweb]

Note: you may need to restart the kernel to use updated packages.


To convert longitude from the $0^{\circ }$ to $360^{\circ }$ range to the $-180^{\circ }$ to $180^{\circ }$ range, 
    use the formula $((longitude + 180) \% 360) - 180$. This formula takes your original longitude, adds $180^{\circ }$, finds 
    the remainder when divided by $360^{\circ }$, and then subtracts $180^{\circ }$ to bring it into the desired range. 

    For more info: https://www.orbiter-forum.com/threads/looking-for-formula-to-convert-0-to-360-to-180-to-180.20767/#:~:text=Assume%20the%20value%20you%20wish,0%20and%20maximum%20of%20360.
    """

In [70]:
def convert_lon(lon):
    # Convert from 0 to 360 range to -180 to 180 range.
    # lons_new_range = []
    # for lon in lons:
    #     new_lon = ((lon+180)%360)-180
    #     lons_new_range.append(new_lon)
    # return lons_new_range
    return ((lon+180)%360)-180

In [ ]:
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap # or Cartopy

def create_map_plot(fig, grid_density):
    # Create a Basemap instance (or Cartopy axes)
    ax = fig.add_subplot(111)
    m = Basemap(projection='mill', llcrnrlat=-90, urcrnrlat=90, \
                llcrnrlon=-180, urcrnrlon=180, resolution='l')
    m.drawcoastlines()
    m.drawcountries()
    m.fillcontinents(color='silver', lake_color='aqua')
    m.drawmapboundary(fill_color='aqua')
    
    # You can add more features like parallels, meridians, or data points here

    # 4. Project the original data to map coordinates
    # lats = np.random.uniform(-90, 90, 1000)
    # lons = np.random.uniform(-180, 180, 1000)
    # vals = np.random.uniform(0, 1000, 1000)

    lats = grid_density['Lat'].values
    lons = grid_density['Lon'].values
    vals = grid_density['Plasma Density'].values
    x, y = m(lons, lats)
    
    # 5. Create the heatmap data using numpy.histogram2d
    #   - You'll need to define the bins for your heatmap grid.
    #   - `weights` can be used if you have a value associated with each point for density calculation.
    # bins = 50
    # heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
    
    # 6. Plot the heatmap using pcolormesh
    #   - `extent` is used to correctly position the heatmap.
    #   - `cmap` defines the color scheme.
    #   - `alpha` controls the transparency, allowing the map features to show through.
    #extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    #m.pcolormesh(xedges, yedges, heatmap.T, cmap='hot_r', alpha=0.7) 
    #m.pcolormesh(x, y, vals, cmap='hot_r', alpha=0.7)
    #plt.colorbar(label='Density') # Add a colorbar for the heatmap
    
    # 7. Overlay the original scatter points
    sc = m.scatter(x, y, s=50, c=vals, alpha=0.5, zorder=5, cmap='plasma') # zorder ensures scatter points are on top

    # Add a colorbar
    cb = fig.colorbar(sc, ax=ax, orientation='vertical', pad=0.02)
    cb.set_label("Plasma Density")

    # Labels
    ax.set_title("Global Plasma Density Heatmap")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    

def map_window(grid_density):
    print("Starting...")
    root = tk.Tk()
    root.title("World Map in Tkinter")
    
    # Create a Matplotlib figure
    fig = Figure(figsize=(8, 6), dpi=100)
    create_map_plot(fig, grid_density)
    
    # Embed the Matplotlib figure into the Tkinter window
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas_widget = canvas.get_tk_widget()
    canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    
    # Add a toolbar for interacting with the plot (zoom, pan, etc.)
    toolbar = NavigationToolbar2Tk(canvas, root)
    toolbar.update()
    canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    
    root.mainloop()

def describe_grid(grid):
    for var in ['Lat', 'Lon', 'Plasma Density']:
        print(f"{var}: {grid[var].min()} - {grid[var].max()}")

# altitudes_m = np.array(ds['Earth_X1']) # m
# colatitudes_rad = np.array(ds['Earth_X2']) # radians
# longitudes_rad = np.array(ds['Earth_X3']) # radians 

# # Convert to desired units.
# altitudes_km = altitudes_m / 1000
# colatitudes_deg = colatitudes_rad * 180/np.pi
# latitudes_deg = 90 - colatitudes_deg
# longitudes_deg = longitudes_rad * 180/np.pi

# grid_density = pd.DataFrame()
# grid_density['Lat'] = latitudes_deg
# grid_density['Lon'] = longitudes_deg
# grid_density['Alt'] = altitudes_km
# grid_density['Plasma Density'] = np.array(ds['Earth_Density'])
# grid_density.head()
# describe_grid(grid_density)


# dd23_3d 
# uncalibrated plasma density in colat-long-time, 1AU surface
altitudes_m = np.array(ds['x_coord']) # m
colatitudes_rad = np.array(ds['y_coord']) # radians
longitudes_rad = np.array(ds['z_coord']) # radians 

# Convert to desired units.
altitudes_km = altitudes_m / 1000
colatitudes_deg = colatitudes_rad * 180/np.pi
latitudes_deg = 90 - colatitudes_deg
longitudes_deg = longitudes_rad * 180/np.pi

vals = np.array(ds['dd23_3d'])
vals.shape # time, lon, colat
# vals_lon_colat = vals[0,:,:]
# vals_lon_colat.shape # lon, colat

longitudes_ls = []
latitudes_ls = []
density_ls = []
for lo, lon in enumerate(longitudes_deg):
    for la, lat in enumerate(latitudes_deg):
        longitudes_ls.append(convert_lon(lon))
        latitudes_ls.append(lat)
        density_ls.append(vals[0,lo,la])

grid_density = pd.DataFrame()
grid_density['Lat'] = latitudes_ls
grid_density['Lon'] = longitudes_ls
#grid_density['Alt'] = altitudes_km
grid_density['Plasma Density'] = density_ls
grid_density.head()
describe_grid(grid_density)

map_window(grid_density)

In [82]:
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas as pd

def create_map_plot(fig, grid_density):
    """
    Create a world map heatmap (scatter plot) using Basemap
    and draw it on the provided Matplotlib figure.
    """

    # Add subplot to the provided figure
    ax = fig.add_subplot(111)

    # Create Basemap on that axis
    m = Basemap(
        projection='mill',
        llcrnrlat=-90, urcrnrlat=90,
        llcrnrlon=-180, urcrnrlon=180,
        resolution='l',
        ax=ax
    )

    # Draw base map
    m.drawcoastlines()
    m.drawcountries()
    m.fillcontinents(color='lightgray', lake_color='lightblue')
    m.drawmapboundary(fill_color='lightblue')

    # Extract data
    lats = grid_density['Lat'].values
    lons = grid_density['Lon'].values
    vals = grid_density['Plasma Density'].values

    # Convert to map projection coords
    x, y = m(lons, lats)

    # Scatter heatmap
    sc = m.scatter(
        x, y,
        s=40,
        c=vals,
        cmap='plasma',
        alpha=0.7,
        edgecolors='none',
        zorder=5
    )

    # Add a colorbar
    cb = fig.colorbar(sc, ax=ax, orientation='vertical', pad=0.02)
    cb.set_label("Plasma Density")

    # Labels
    ax.set_title("Global Plasma Density Heatmap")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")


def map_window(grid_density):
    """Create a Tkinter popup window containing the world heatmap."""
    root = tk.Tk()
    root.title("World Heatmap Viewer")

    # Create the figure ONLY once
    fig = Figure(figsize=(9, 6), dpi=100)
    create_map_plot(fig, grid_density)

    # Embed inside Tkinter
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas_widget = canvas.get_tk_widget()
    canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    # Add toolbar
    toolbar = NavigationToolbar2Tk(canvas, root)
    toolbar.update()
    toolbar.pack(side=tk.TOP, fill=tk.X)

    root.mainloop()


# Example usage:
# grid_density = pd.DataFrame({
#     "Lat": latitudes_ls,
#     "Lon": longitudes_ls,
#     "Plasma Density": density_ls
# })
map_window(grid_density)


In [ ]:
# import tkinter as tk
# from matplotlib.figure import Figure
# from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
# import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap # or Cartopy

# def create_map_plot(fig, grid_density):
#     # Create a Basemap instance (or Cartopy axes)
#     # Example using Basemap:
#     fig = plt.figure(figsize=(8, 6))
#     m = Basemap(projection='mill', llcrnrlat=-90, urcrnrlat=90, \
#                 llcrnrlon=-180, urcrnrlon=180, resolution='l', ax=fig.add_subplot(111))
#     m.drawcoastlines()
#     m.drawcountries()
#     m.fillcontinents(color='silver', lake_color='aqua')
#     m.drawmapboundary(fill_color='aqua')

#     lats = grid_density['Lat']
#     lons = grid_density['Lon']
#     vals = grid_density['Plasma Density']
#     x, y = m(lons, lats)
    
#     # 7. Overlay the original scatter points
#     m.scatter(x, y, s=50, c=vals, alpha=0.5, zorder=5, cmap='plasma') # zorder ensures scatter points are on top

#     # Add x and y labels
#     plt.xlabel('Longitude', labelpad=40)  # labelpad adjusts distance from axis
#     plt.ylabel('Latitude', labelpad=40)
    

# def map_window(grid_density):
#     print("Starting...")
#     root = tk.Tk()
#     root.title("World Map in Tkinter")
    
#     # Create a Matplotlib figure
#     fig = Figure(figsize=(8, 6), dpi=100)
#     create_map_plot(fig, grid_density)
    
#     # Embed the Matplotlib figure into the Tkinter window
#     canvas = FigureCanvasTkAgg(fig, master=root)
#     canvas_widget = canvas.get_tk_widget()
#     canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    
#     # Add a toolbar for interacting with the plot (zoom, pan, etc.)
#     toolbar = NavigationToolbar2Tk(canvas, root)
#     toolbar.update()
#     canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    
#     root.mainloop()

# grid_density = pd.DataFrame()
# grid_density['Lat'] = latitudes_ls
# grid_density['Lon'] = longitudes_ls
# grid_density['Plasma Density'] = density_ls
# grid_density.head()

# map_window(grid_density)

In [55]:
len(ds['time'])

169

In [63]:
# grid_density['Lat'] = latitudes_deg
# grid_density['Lon'] = longitudes_deg
# grid_density['Alt'] = altitudes_km
# grid_density['Plasma Density'] = np.array(ds['dd23_3d'])



In [65]:
len(longitudes_ls)

10800

In [67]:
grid_density.describe()

,Lat,Lon,Plasma Density
count,10800.000000,10800.000000,10800.000000
mean,0.000002,180.000000,-22441.132130
std,34.637806,103.921707,9090.914059
min,-59.000000,1.000000,-32648.000000
25%,-29.499992,90.499994,-29253.250000
50%,0.000004,180.000000,-25282.000000
75%,29.500000,269.499992,-18371.750000
max,59.000000,359.000000,31047.000000


In [2]:
# pip install basemap

   ---------------------------------------- 0.0/30.5 MB ? eta -:--:--
   --- ------------------------------------ 2.4/30.5 MB 10.6 MB/s eta 0:00:03
   ------ --------------------------------- 5.2/30.5 MB 14.8 MB/s eta 0:00:02
   ------------ --------------------------- 9.4/30.5 MB 15.3 MB/s eta 0:00:02
   ---------------- ----------------------- 12.6/30.5 MB 16.9 MB/s eta 0:00:02
   -------------------- ------------------- 16.0/30.5 MB 15.6 MB/s eta 0:00:01
   -------------------------- ------------- 19.9/30.5 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 19.9/30.5 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 19.9/30.5 MB 16.2 MB/s eta 0:00:01
   -------------------------------- ------- 25.2/30.5 MB 13.8 MB/s eta 0:00:01
   ------------------------------------- -- 28.3/30.5 MB 14.0 MB/s eta 0:00:01
   ---------------------------------------- 30.5/30.5 MB 13.7 MB/s eta 0:00:00

   -------------------------- ------------- 2/3 [basemap]
   --

In [19]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
# Find directories (typically they end with '/')



In [20]:
directories

['2011/',
 '2012/',
 '2013/',
 '2014/',
 '2015/',
 '2016/',
 '2017/',
 '2018/',
 '2019/',
 '2020/',
 '2021/',
 '2022/',
 '2023/',
 '2024/',
 '2025/']

In [85]:
file = r"C:\Users\haley\Documents\Code\cole-tech\Data\swpc_wsaenlil_bkg_20251004_0000\wsa_enlil.mrid00000000.inputs\grd.nc"
data = read_nc(file)


Dataset Information:
xarray.Dataset {
dimensions:
	nblk = 1 ;
	n1 = 512 ;
	n2 = 60 ;
	n3 = 180 ;
	n1h = 513 ;
	n2h = 61 ;
	n3h = 181 ;

variables:
	float64 X1(nblk, n1) ;
		X1:long_name = X1-cell positions ;
		X1:units = m ;
	float64 X2(nblk, n2) ;
		X2:long_name = X2-cell positions ;
		X2:units = radian ;
	float64 X3(nblk, n3) ;
		X3:long_name = X3-cell positions ;
		X3:units = radian ;
	float64 X1H(nblk, n1h) ;
		X1H:long_name = X1-interface positions ;
		X1H:units = m ;
	float64 X2H(nblk, n2h) ;
		X2H:long_name = X2-interface positions ;
		X2H:units = radian ;
	float64 X3H(nblk, n3h) ;
		X3H:long_name = X3-interface positions ;
		X3H:units = radian ;

// global attributes:
	:type = GRD ;
	:title = Grid coordinates ;
	:program = reg2grd ;
	:version = 2.9 ;
	:project = a8b1_newer/2011_11_27 ;
	:code =  ;
	:model =  ;
	:geometry = spherical ;
	:grid = X1=0.1-1.7/uniform X2=30-150/uniform X3=0-360/uniform ;
	:rotation =  ;
	:case = reg2med1.dvb-a8b1-d4t1x1.20111127T00 ;
	:cordata =  ;
